### Foursquare Credentials:

In [ ]:
CLIENT_ID = 'QVVZFBOIM23WXTHS0QNXXXQRKDR2Y0RZ1IVUKQ5SXP013CWI' # your Foursquare ID
CLIENT_SECRET = 'AQNG4XNAKC025K3OASFTQR4WVQGP4LGFRLJ5FPNC4W1E0VEK' # your Foursquare Secret
VERSION = '20181018'
LIMIT = 30
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:  ' + CLIENT_SECRET)
print('VERSION:  ' + VERSION)
print('LIMIT: ' , LIMIT)

### Get latitude & longitutde from City Address:

In [ ]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
LATITUDE = location.latitude
LONGITUDE = location.longitude
print("Address : ", address)
print("LATITUDE : ", LATITUDE)
print("LONGITUDE : ", LONGITUDE)

### Search a Specific Venue Category:

In [ ]:
SEARCH_QUERY = "Italian"
RADIUS = 500 # meter
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, LATITUDE, LONGITUDE, VERSION, SEARCH_QUERY, RADIUS, LIMIT)


In [ ]:
request = requests.get(url).json()

In [ ]:
with open('venues_data.json', 'w', encoding='utf-8') as f:
    json.dump(request, f, ensure_ascii=False, indent=4)
    
venues = request['response']['venues']
print("Total Venues :  ", len(venues))
df_venues =  json_normalize(venues)
print("Total Venue fields : ", len(list(df_venues.columns)))

### Filter (Preprocess) the data:

In [ ]:
venues = request['response']['venues']
print("Total Venues = ", len(venues))
df_venues = json_normalize(venues)

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_venues.columns if col.startswith('location.')] + ['id']
df_venues_filtered = df_venues.loc[:, filtered_columns] 
# filter the category for each row
df_venues_filtered['categories'] = df_venues_filtered.apply(get_category_type, axis=1)
# clean column names by keeping only last term
df_venues_filtered.columns = [column.split('.')[-1] for column in df_venues_filtered.columns]
# df_venues_filtered

In [ ]:
# Nearby restaurants:
df_venues_filtered.name

### Visualize and Explore nearby Resturants:

In [ ]:
venues_map = folium.Map(location=[LATITUDE, LONGITUDE], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [LATITUDE, LONGITUDE],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(df_venues_filtered.lat, df_venues_filtered.lng, df_venues_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Lets Explore a Venue:

### 1. Exploring closest Italian Resturant:

In [ ]:
df_venues_filtered

In [ ]:
# Get id from database from resturant category:  
VENUE_ID = df_venues_filtered[df_venues_filtered['categories']=="Italian Restaurant"]['id'][2]
VENUE_ID

In [ ]:
venue_id = VENUE_ID # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
print('url : ', url)

In [ ]:
request = requests.get(url).json()

In [ ]:
with open('Italian Pizza.json', 'w', encoding='utf-8') as f:
    json.dump(request, f, ensure_ascii=False, indent=4)

In [ ]:
df_columns =  list(request['response']['venue'].keys())
rating = request['response']['venue']['rating']
tips = request['response']['venue']['tips']['count']

### 2. Lets Get Venue Tips:

In [ ]:
tips = request['response']['venue']['tips']['count']
limit = tips = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

In [ ]:
request = requests.get(url).json()

In [ ]:
with open('Tips.json', 'w', encoding='utf-8') as f:
    json.dump(request, f, ensure_ascii=False, indent=4)

In [ ]:
tips = request['response']['tips']['items']
tip = request['response']['tips']['items'][0]
tip.keys()

In [ ]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]
# display tips
tips_df

### 3. Lets Get Foursquare User:

In [ ]:
user_id = '484542633' # user ID with most agree counts and complete profile
url = "https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}".format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL


"https://foursquare.com/oauth2/authenticate?client_id={}&response_type=code&redirect_uri=https://www.google.com"

In [ ]:
# send GET request
request = requests.get(url).json()
request

#### Lets postpond it due to invalid user_id

### 4. Lets Get Location:

In [ ]:
latitude = 40.715337
longitude = -74.008848
radius = 500

In [ ]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

In [ ]:
results = requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))

In [ ]:
items = results['response']['groups'][0]['items']

In [ ]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

In [ ]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map